### Model Inference with InterpretML

In [ ]:
import pandas as pd
import numpy as np
from functions import df_stats
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import time

import altair as alt
from vega_datasets import data
#alt.renderers.enable('notebook')
alt.data_transformers.disable_max_rows()

#### Data Set is large, a smaller random sample will be sufficient for tests

In [ ]:
SEED=0 # set random state

path = 'data/'
df_raw = pd.read_csv(path+'train.csv').sample(100000, random_state=SEED)
df_raw.shape

In [ ]:
def df_stats(df):
    
    stats = pd.DataFrame(index=list(df))
    stats['DataTypes'] = df.dtypes
    stats['MissingPct'] = df.isnull().sum()/df.shape[0]*100
    stats['NUnique'] = df.nunique().astype(float)
    stats.reset_index().to_excel('stats.xlsx')
    print(stats)

In [ ]:
df_stats(df_raw)

## Features at hand:
* `ID` is completely unique and object type, it won't be used;
* `datetime` contains detailed time and can be preprocessed in different ways to maximize its utility;
* `siteid`, `offerid`, `category`, `merchant` will be used as numeric columns (even though they actually refer to high cardinality categories);
* `countrycode`, `browserid`, `devid` are categorical and therefore will be converted to numeric.
* `click` is the target variable which will be classified. 1 refers to instances where it was clicked.

In [ ]:
# Drop ID column
df_raw.drop("ID", axis=1, inplace=True)

### Data Preprocessing and EDA

#### Split data to train (70%) and holdout (30%) sets

In [ ]:
X_t, X_h, y_t, y_h = train_test_split(
    df_raw.drop('click',axis=1),
    df_raw['click'],
    test_size=0.3,
    random_state=SEED
)
X_t.shape, X_h.shape

In [ ]:
y_t.value_counts(normalize=True)

In [ ]:
y_h.value_counts(normalize=True)

### Numerical Features

In [ ]:
#themes = ['dark', 'default', 'fivethirtyeight', 'ggplot2', 'latimes', 'none', 'opaque', 'quartz', 'vox']
alt.themes.enable('vox')

y = 'click'
df_temp = X_t.copy()
df_temp[y] = y_t

for x in ['offerid', 'category', 'merchant', 'siteid']:

    chart = alt.Chart(df_temp).transform_density(
        density=x,
        as_=[x,'density'],
        groupby=[y],
        counts = False
    ).mark_area(fillOpacity=0.3).encode(
        x=x+':Q',
        y='density:Q',
        color=alt.Color(y+':N', scale=alt.Scale(scheme='set1')),
        stroke=y+':N'
    ).properties(width=800, height=200)
    display(chart)

### Categorical features can be visualized with a heatmap
* https://altair-viz.github.io/gallery/layered_heatmap_text.html

In [ ]:
cols = ['countrycode', 'browserid', 'devid']
y = 'click'

for x in cols:
    
    to_plot = pd.crosstab(df_temp[x], df_temp[y]).reset_index(drop=False).melt(id_vars=[x])
    size_t, size_h = tuple(df_temp[y].value_counts())
    to_plot['rel_prop'] = round(to_plot.apply(
        lambda col: col['value']/size_t if col['click']==0 else col['value']/size_h, axis=1),2)
    
    base = alt.Chart(to_plot).encode(
    x=x, y='click:O'
    ).properties(width=700, height=200)

    heatmap = base.mark_rect().encode(
        color=alt.Color('rel_prop:Q', scale=alt.Scale(scheme='viridis'))
    )

    text = base.mark_text().encode(
        text='rel_prop:Q'
    )
    display(heatmap + text)

#### `countrycode`

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
freq_table = pd.crosstab(df_raw['countrycode'], df_raw['click'], margins=True, dropna=False)
freq_table/df_raw.shape[0]*100

`countrycode` "c" and "d" may have some predictive power, as their overall frequency is rather low, but they are  clicked significantly more often than other codes.

`devid`

In [ ]:
freq_table = pd.crosstab(df_raw['devid'], df_raw['click'], margins=True, dropna=False)
freq_table/df_raw.shape[0]*100

#### It turns out that all of the dates are in year 2017, since year variable would have no variance, lets drop it.

There are some categorical features remaining which require encoding, for that we can use ordinal encoder.
As much as I like to use sklearn, its class OrdinalEncoder doesn't accept missing values or out of sample encoding (while transforming), so an alternative library called category_encoders is used instead.

### Model Pipeline

* we will skip hyper parameter tuning and cross validation for computation reasons, but it can be easily included with sklearn classes `GridSearchCV` or `RandomSearchCV`

In [ ]:
from sklearn.pipeline import Pipeline
from category_encoders.ordinal import OrdinalEncoder
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
import shap
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

In [ ]:
class DateTransformer(BaseEstimator, TransformerMixin):

    def __init__(self,use_dates = ['year', 'month', 'day', 'hour', 'minute', 'second']):
        self._use_dates = use_dates
        
    def fit(self, X, y = None):
        return self

    def get_dates(self, obj):        
        return pd.DatetimeIndex(obj)
    
    def transform(self, X , y = None ):

        for spec in self._use_dates:
            
            command = f"X['{spec}'] = self.get_dates(X['datetime']).{spec}"
            exec(command)
        
        X = X.drop(['datetime','year'], axis = 1)

        return X

* Test `DateTransformer()` Class;
* Save column list (pipeline returns numpy array and doesn't remember column names);
* Test `OrdinalEncoder()`

In [ ]:
df_sample = X_t.head(500).copy()
ct = DateTransformer()
ct.fit(X=df_sample, y = None)
a = ct.transform(df_sample)
feature_names = list(a)
a

In [ ]:
enc = OrdinalEncoder(handle_unknown='value', handle_missing='value')
d = a[['countrycode','devid', 'browserid']].head(10)
enc.fit(d)
enc.transform(d)

#### Define and fit pipelines

In [ ]:
# lgbm pipeline
pipeline_lgb = Pipeline([
    ("transform_dates", DateTransformer()),
    ("transform_cats", OrdinalEncoder(handle_unknown='value', handle_missing='value')),
    ("impute", SimpleImputer(fill_value=0)),
    ("model", LGBMClassifier())
    ])

In [ ]:
start = pd.Timestamp.now()
pipeline_lgb.fit(X=X_t, y=y_t)
print(pd.Timestamp.now()-start)

In [ ]:
start = pd.Timestamp.now()
predictions_lgb = pipeline_lgb.predict(X=X_h)
print(pd.Timestamp.now()-start)

`siteid, offerid, category, merchant`

* Pipeline can't return missing values, because they are not supported by explainable ML

In [ ]:
# EBM pipeline
start = pd.Timestamp.now()
pipeline_ebm = Pipeline([
    ("transform_dates", DateTransformer()),
    ("transform_cats", OrdinalEncoder(handle_unknown='value', handle_missing='value')),
    ("impute", SimpleImputer(fill_value=0)),
    ("model", ExplainableBoostingClassifier(feature_names=feature_names))
    ])

pipeline_ebm.fit(X_t, y_t)
print(pd.Timestamp.now()-start)

In [ ]:
start = pd.Timestamp.now()
predictions_ebm = pipeline_ebm.predict(X=X_h)
print(pd.Timestamp.now()-start)

#### accuracy

In [ ]:
print(classification_report(y_h, predictions_lgb))

In [ ]:
print(classification_report(y_t, pipeline_ebm.predict(X=X_t)))

#### Add accuracy results from both models on training and test sets to a single table

In [ ]:
train_lgb_report = pd.DataFrame(classification_report(y_t, pipeline_lgb.predict(X=X_t), output_dict=True))
test_lgb_report = pd.DataFrame(classification_report(y_h, predictions_lgb, output_dict=True))
lgb_report = pd.concat([train_lgb_report,test_lgb_report])[["0", "1"]]

train_ebm_report = pd.DataFrame(classification_report(y_t, pipeline_ebm.predict(X=X_t), output_dict=True))
test_ebm_report = pd.DataFrame(classification_report(y_h, predictions_ebm, output_dict=True))
ebm_report = pd.concat([train_ebm_report,test_ebm_report])[["0", "1"]]

full_report = pd.concat([lgb_report, ebm_report], axis=1)

index = list(full_report.index)
multi_index = []

for i in range(0, len(index)):
    if i < 4:
        multi_index.append(f'training set: {index[i]}')
    else:
        multi_index.append(f'holdout set: {index[i]}')

full_report.index = multi_index
full_report.columns = ['0 (lgb)', '1 (lgb)', '0 (ebm)', '1 (ebm)']
full_report

This is an imbalanced classification task and the majority class (0) are nearly perfectly predicted, and minority class (1) is predicted reasonably well.

### Model interpretation
When building an interpretable we need to think about how we will construct our pipeline as well. Model inference libraries are not suited for all inputs.
* SHAP tree explainer supports missing values but doesn't support sklearn Pipeline or non numeric values. When building a gradient boosting model, tree explainer lets use the benefits of computing shap values fast.
* Interpret_ML doesn't support missing values, but can be fitted on the sklearn Pipeline. Does it support categorical features?

The pipeline consists of three steps, but we only need the first two, which transform data: 

This library also provides tools like `ClassHistogram`, which can be useful to conduct EDA, but you may need to sample data. Since it's interactive and runs under `Plotly` it may quickly become something that takes long to load as data grows.

In [ ]:
# Create data sets with pipeline preprocessing
X_t_prep = pd.DataFrame(data=pipeline_lgb[0:3].transform(X_t), columns=feature_names)
X_h_prep = pd.DataFrame(data=pipeline_lgb[0:3].transform(X_h), columns=feature_names)

In [ ]:
from interpret import show
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X_t_prep, y_t, name = 'Train Data')
show(hist)

In [ ]:
# SHAP
start = pd.Timestamp.now()
explainer = shap.TreeExplainer(pipeline_lgb['model'])
shap_values = explainer.shap_values(X_t_prep)
print(pd.Timestamp.now()-start)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 90

ebm_global = pipeline_ebm['model'].explain_global()
show(ebm_global)

shap.summary_plot(shap_values[0], X_t_prep, plot_type="bar", plot_size=(10,5))

In [ ]:
show(ebm_global)
shap.dependence_plot(ind="countrycode", shap_values=shap_values[0], features=X_t_prep, interaction_index=None)

In [ ]:
ind = [69950]
# InterpretML
print(f"True value: {y_t.iloc[ind]}")
ebm_local = pipeline_ebm['model'].explain_local(X_t_prep.iloc[ind], y_t.iloc[ind], name='Local')
show(ebm_local)
# SHAP
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0][ind,:], X_t_prep.iloc[ind])

In [ ]:
pd.DataFrame(shap_values[0][ind,:], columns = list(X_t_prep))

In [ ]:
X_t_prep.iloc[ind]